# Data Analysis Codes

In [1]:
### Final Data Ready for Propensity Score Matching:   "/work/campers_ipv_for_analysis_employment.csv"

## Matching

## Log-Linear Regression Analysis

Dataset

In [8]:
ipv <- read.csv("campers_ipv_for_analysis_employment.csv")

Reclassifying variables

In [9]:
ipv$Marital <- factor(ipv$Marital)
ipv$Husb.Educ.Level <- factor(ipv$Husb.Educ.Level)
ipv$Region <- factor(ipv$Region)
ipv$LPG <- relevel(factor(ipv$LPG), ref = "Male only Employed")

Descriptives

In [7]:
aggregate(ipv$Counts.IPV, list(ipv$LPG), FUN = mean) 
aggregate(ipv$Counts.IPV, list(ipv$LPG), FUN = sum)
aggregate(ipv$Counts.IPV, list(ipv$LPG), FUN = length)
table(ipv$Counts.IPV)

Group.1,x
<fct>,<dbl>
Male only Employed,0.3708029
Both employed,0.4319030
Both unemployed,0.4864865
Female only Employed,0.6119403


Group.1,x
<fct>,<int>
Male only Employed,2286
Both employed,2315
Both unemployed,54
Female only Employed,82


Group.1,x
<fct>,<int>
Male only Employed,6165
Both employed,5360
Both unemployed,111
Female only Employed,134



   0    1    2    3    9   10   19 
8613 1929  905  320    1    1    1 

Poisson Fit

In [6]:
ipv.pois <- glm(Counts.IPV ~ LPG + Husb.Educ.Level + Region + Age 
                    + Marital, family = poisson, data = ipv)
summary(ipv.pois)


Call:
glm(formula = Counts.IPV ~ LPG + Husb.Educ.Level + Region + Age + 
    Marital, family = poisson, data = ipv)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.4790  -0.9217  -0.7801   0.4389   9.9272  

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -1.503515   0.153062  -9.823  < 2e-16 ***
LPGBoth employed         0.181990   0.030277   6.011 1.85e-09 ***
LPGBoth unemployed       0.329543   0.138476   2.380 0.017323 *  
LPGFemale only Employed  0.649781   0.113157   5.742 9.34e-09 ***
Husb.Educ.Level1         0.433532   0.117757   3.682 0.000232 ***
Husb.Educ.Level2         0.350789   0.117715   2.980 0.002883 ** 
Husb.Educ.Level3         0.051484   0.120439   0.427 0.669035    
Region2                  0.188654   0.100525   1.877 0.060560 .  
Region3                 -0.057480   0.092677  -0.620 0.535115    
Region4                  0.205507   0.099471   2.066 0.038829 *  
Region5                  0.2

In [7]:
# Intercept-only (Poisson)
ipv.int <- glm(Counts.IPV ~ 1, family = poisson, data = ipv)

Diagnostics of the Poisson Fit

In [8]:
# Goodness-of-Fit

# install.packages("lmtest")
library(lmtest)
lrtest(ipv.int, ipv.pois)

Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric




,#Df,LogLik,Df,Chisq,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,-10316.29,NA,NA,NA
2,27,-10032.31,26,567.9581,2.800533e-103


In [9]:
# Multicollinearity

# install.packages("regclass")
library(regclass)
VIF(ipv.pois)

Loading required package: bestglm

Loading required package: leaps

Loading required package: VGAM

Loading required package: stats4

Loading required package: splines


Attaching package: ‘VGAM’


The following object is masked from ‘package:lmtest’:

    lrtest


Loading required package: rpart

Loading required package: randomForest

randomForest 4.7-1.1

Type rfNews() to see new features/changes/bug fixes.

Important regclass change from 1.3:
All functions that had a . in the name now have an _
all.correlations -> all_correlations, cor.demo -> cor_demo, etc.




,GVIF,Df,GVIF^(1/(2*Df))
LPG,1.078564,3,1.012685
Husb.Educ.Level,1.074762,3,1.012089
Region,1.116973,18,1.003078
Age,1.152788,1,1.073680
Marital,1.151514,1,1.073086


In [11]:
# Test for Overdispersion
mean(ipv$Counts.IPV)
var(ipv$Counts.IPV)

install.packages("AER")
library(AER)
dispersiontest(ipv.pois, alternative = "greater")

# for some reason, di gumagana AER dito. pero significant yung dispersion test sa R, 
# thus overdispersed

[1] 0.4024639

[1] 0.6002658

Installing package into ‘/work/.R/library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘nloptr’, ‘pbkrtest’, ‘lme4’, ‘car’


Warning message in install.packages("AER"):
“installation of package ‘nloptr’ had non-zero exit status”
Warning message in install.packages("AER"):
“installation of package ‘lme4’ had non-zero exit status”
Warning message in install.packages("AER"):
“installation of package ‘pbkrtest’ had non-zero exit status”
Warning message in install.packages("AER"):
“installation of package ‘car’ had non-zero exit status”
Warning message in install.packages("AER"):
“installation of package ‘AER’ had non-zero exit status”


ERROR: Error in library(AER): there is no package called ‘AER’


Negative Binomial Fit

In [12]:
install.packages("MASS")
library(MASS)

Installing package into ‘/work/.R/library’
(as ‘lib’ is unspecified)



In [13]:
ipv.nb <- glm.nb(Counts.IPV ~ LPG + Husb.Educ.Level + Region + Age + Marital,
                     data = ipv)
summary(ipv.nb)


Call:
glm.nb(formula = Counts.IPV ~ LPG + Husb.Educ.Level + Region + 
    Age + Marital, data = ipv, init.theta = 0.8588779575, link = log)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.1864  -0.8297  -0.7208   0.3151   5.4718  

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -1.512971   0.182028  -8.312  < 2e-16 ***
LPGBoth employed         0.179451   0.037113   4.835 1.33e-06 ***
LPGBoth unemployed       0.339255   0.174721   1.942  0.05217 .  
LPGFemale only Employed  0.635658   0.150026   4.237 2.27e-05 ***
Husb.Educ.Level1         0.432493   0.138499   3.123  0.00179 ** 
Husb.Educ.Level2         0.350171   0.138368   2.531  0.01138 *  
Husb.Educ.Level3         0.048959   0.141381   0.346  0.72912    
Region2                  0.200260   0.119728   1.673  0.09440 .  
Region3                 -0.048311   0.108817  -0.444  0.65706    
Region4                  0.222962   0.119309   1.869  0.06165 .  
Regi

Variable Selection (Backward Elimination)

In [14]:
# no marital
ipv.nb_no.marital <- glm.nb(Counts.IPV ~ LPG + Husb.Educ.Level + Region + Age,
                            data = ipv)
summary(ipv.nb_no.marital)
anova(ipv.nb, ipv.nb_no.marital)


Call:
glm.nb(formula = Counts.IPV ~ LPG + Husb.Educ.Level + Region + 
    Age, data = ipv, init.theta = 0.8313925637, link = log)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.1943  -0.8363  -0.7318   0.3520   5.7154  

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -1.164517   0.178009  -6.542 6.07e-11 ***
LPGBoth employed         0.174266   0.037233   4.680 2.86e-06 ***
LPGBoth unemployed       0.413127   0.174617   2.366  0.01799 *  
LPGFemale only Employed  0.644826   0.150539   4.283 1.84e-05 ***
Husb.Educ.Level1         0.434234   0.139055   3.123  0.00179 ** 
Husb.Educ.Level2         0.362732   0.138919   2.611  0.00902 ** 
Husb.Educ.Level3         0.039443   0.141924   0.278  0.78108    
Region2                  0.126549   0.119598   1.058  0.29000    
Region3                 -0.075478   0.108929  -0.693  0.48837    
Region4                  0.199423   0.119885   1.663  0.09622 .  
Region5       

Model,theta,Resid. df,2 x log-lik.,Test,df,LR stat.,Pr(Chi)
<chr>,<dbl>,<int>,<dbl>,<chr>,<int>,<dbl>,<dbl>
LPG + Husb.Educ.Level + Region + Age,0.8313926,11744,-19443.20,,NA,NA,NA
LPG + Husb.Educ.Level + Region + Age + Marital,0.8588780,11743,-19373.67,1 vs 2,1,69.53009,1.110223e-16


In [15]:
# no educ
ipv.nb_no.educ <- glm.nb(Counts.IPV ~ LPG + Region + Age + Marital,
                         data = ipv)
summary(ipv.nb_no.educ)
anova(ipv.nb, ipv.nb_no.educ)


Call:
glm.nb(formula = Counts.IPV ~ LPG + Region + Age + Marital, data = ipv, 
    init.theta = 0.83017846, link = log)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.1953  -0.8247  -0.7329   0.3346   5.6706  

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -1.232150   0.117857 -10.455  < 2e-16 ***
LPGBoth employed         0.167227   0.037127   4.504 6.66e-06 ***
LPGBoth unemployed       0.249122   0.174993   1.424 0.154559    
LPGFemale only Employed  0.553822   0.150116   3.689 0.000225 ***
Region2                  0.195255   0.119881   1.629 0.103368    
Region3                 -0.029430   0.108937  -0.270 0.787041    
Region4                  0.236417   0.119336   1.981 0.047579 *  
Region5                  0.332485   0.101240   3.284 0.001023 ** 
Region6                  0.062280   0.104198   0.598 0.550036    
Region7                  0.619487   0.105360   5.880 4.11e-09 ***
Region8                 

Model,theta,Resid. df,2 x log-lik.,Test,df,LR stat.,Pr(Chi)
<chr>,<dbl>,<int>,<dbl>,<chr>,<int>,<dbl>,<dbl>
LPG + Region + Age + Marital,0.8301785,11746,-19441.56,,NA,NA,NA
LPG + Husb.Educ.Level + Region + Age + Marital,0.8588780,11743,-19373.67,1 vs 2,3,67.89291,1.210143e-14


In [16]:
# no region
ipv.nb_no.region <- glm.nb(Counts.IPV ~ LPG + Husb.Educ.Level + Age + Marital,
                 data = ipv)
summary(ipv.nb_no.region)
anova(ipv.nb, ipv.nb_no.region)


Call:
glm.nb(formula = Counts.IPV ~ LPG + Husb.Educ.Level + Age + Marital, 
    data = ipv, init.theta = 0.7907103814, link = log)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.1431  -0.8202  -0.7495   0.3831   5.2484  

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -1.276135   0.162558  -7.850 4.15e-15 ***
LPGBoth employed         0.203964   0.037108   5.497 3.87e-08 ***
LPGBoth unemployed       0.262106   0.177261   1.479  0.13923    
LPGFemale only Employed  0.613202   0.151027   4.060 4.90e-05 ***
Husb.Educ.Level1         0.418917   0.137880   3.038  0.00238 ** 
Husb.Educ.Level2         0.317619   0.137255   2.314  0.02066 *  
Husb.Educ.Level3         0.017361   0.140159   0.124  0.90142    
Age                     -0.004592   0.002840  -1.617  0.10597    
Marital2                 0.365455   0.040201   9.091  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion

Model,theta,Resid. df,2 x log-lik.,Test,df,LR stat.,Pr(Chi)
<chr>,<dbl>,<int>,<dbl>,<chr>,<int>,<dbl>,<dbl>
LPG + Husb.Educ.Level + Age + Marital,0.7907104,11761,-19549.91,,NA,NA,NA
LPG + Husb.Educ.Level + Region + Age + Marital,0.8588780,11743,-19373.67,1 vs 2,18,176.2474,0


In [17]:
# no age
ipv.nb_no.age <- glm.nb(Counts.IPV ~ LPG + Husb.Educ.Level + Region + Marital,
                 data = ipv)
summary(ipv.nb_no.age)
anova(ipv.nb, ipv.nb_no.age)


Call:
glm.nb(formula = Counts.IPV ~ LPG + Husb.Educ.Level + Region + 
    Marital, data = ipv, init.theta = 0.8574066954, link = log)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.1754  -0.8317  -0.7217   0.3240   5.5577  

Coefficients:
                        Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -1.67404    0.15774 -10.613  < 2e-16 ***
LPGBoth employed         0.16836    0.03662   4.598 4.27e-06 ***
LPGBoth unemployed       0.34459    0.17475   1.972  0.04861 *  
LPGFemale only Employed  0.62444    0.15005   4.162 3.16e-05 ***
Husb.Educ.Level1         0.43985    0.13854   3.175  0.00150 ** 
Husb.Educ.Level2         0.36236    0.13832   2.620  0.00880 ** 
Husb.Educ.Level3         0.05890    0.14137   0.417  0.67697    
Region2                  0.20433    0.11971   1.707  0.08785 .  
Region3                 -0.04654    0.10880  -0.428  0.66885    
Region4                  0.22524    0.11930   1.888  0.05903 .  
Region5              

Model,theta,Resid. df,2 x log-lik.,Test,df,LR stat.,Pr(Chi)
<chr>,<dbl>,<int>,<dbl>,<chr>,<int>,<dbl>,<dbl>
LPG + Husb.Educ.Level + Region + Marital,0.8574067,11744,-19376.75,,NA,NA,NA
LPG + Husb.Educ.Level + Region + Age + Marital,0.8588780,11743,-19373.67,1 vs 2,1,3.088018,0.07887081


All factors contribute significantly to the model. The final model is the NB model.

In [18]:
# Intercept-only (NB)
ipv.int.nb <- glm.nb(Counts.IPV ~ 1, data = ipv)

Diagnostics of the Final Fit

In [19]:
# Goodness-of-Fit
anova(ipv.nb, ipv.int.nb) # pwede ring lrtest() pero di gumagana here

Model,theta,Resid. df,2 x log-lik.,Test,df,LR stat.,Pr(Chi)
<chr>,<dbl>,<int>,<dbl>,<chr>,<int>,<dbl>,<dbl>
1,0.7131244,11769,-19753.07,,NA,NA,NA
LPG + Husb.Educ.Level + Region + Age + Marital,0.8588780,11743,-19373.67,1 vs 2,26,379.4073,0


In [20]:
# Multicollinearity
VIF(ipv.nb)

,GVIF,Df,GVIF^(1/(2*Df))
LPG,1.075637,3,1.012226
Husb.Educ.Level,1.077452,3,1.012511
Region,1.117797,18,1.003098
Age,1.147042,1,1.071001
Marital,1.146648,1,1.070817


## Logistic Regression Analysis (try lang)

Data Preparation

In [21]:
ipv3 <- ipv
val <- c(2, 3, 9, 10, 19)

ipv3["Counts.IPV"] <- sapply(ipv3["Counts.IPV"], 
                             function(x) replace(x, x %in% val, 1))

Logistic Fit

In [22]:
ipv3.log <- glm(Counts.IPV ~ LPG + Husb.Educ.Level + Region + Age + Marital,
                family = binomial, data = ipv3)
summary(ipv3.log) # pangit pa rin


Call:
glm(formula = Counts.IPV ~ LPG + Husb.Educ.Level + Region + Age + 
    Marital, family = binomial, data = ipv3)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.2338  -0.8231  -0.6903   1.3195   2.1345  

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -1.681942   0.214775  -7.831 4.83e-15 ***
LPGBoth employed         0.216966   0.044025   4.928 8.30e-07 ***
LPGBoth unemployed       0.156737   0.220469   0.711 0.477130    
LPGFemale only Employed  0.697922   0.186674   3.739 0.000185 ***
Husb.Educ.Level1         0.512921   0.162791   3.151 0.001628 ** 
Husb.Educ.Level2         0.418788   0.162556   2.576 0.009987 ** 
Husb.Educ.Level3         0.148140   0.165686   0.894 0.371268    
Region2                  0.205771   0.142870   1.440 0.149792    
Region3                  0.043819   0.127338   0.344 0.730763    
Region4                  0.254721   0.142592   1.786 0.074040 .  
Region5                  0

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8672f010-76e5-4fe5-a087-2513d801c0e9' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>